In [1]:
%cd ..

/Users/alexo/Documents/projects/llama3_exploration_project


/Users/alexo/miniconda3/envs/llama_exploration/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import time
import torch
from transformers import AutoTokenizer
from utils.utils import load_json

/Users/alexo/miniconda3/envs/llama_exploration/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare text and embeddings

In [3]:
device="mps"
model_dir = "converted_models/GRANITE-3B-CODE-INSTRUCT-PKL"
config = load_json(f"./{model_dir}/config.json")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, clean_up_tokenization_spaces=False)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def text_to_ids(text, tokenizer):
    if type(text) != list: text = [text]
    input_ids = tokenizer(
        text,
    )["input_ids"]
    return input_ids

## Forward passes

In [6]:
from ops.transformer_ops import Transformer
from ops.generation import generate_text, generate_text_stream

In [7]:
config["model_type"] = "granite-small"

In [8]:
model = Transformer(model_dir, config, device=device)

In [9]:
chat = [
    { "role": "user", "content": "Write a code to find the maximum value in a list of numbers." },
]
input_ids = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True)
# tokenize the text
input_ids = [input_ids]

In [10]:
streaming=True

In [11]:
if streaming:
    output_text = []
    total_tokens_count = 0
    start_time = time.time()
    for word, n_tokens in generate_text_stream(model, tokenizer, input_ids, max_gen_len=128, stop_tokens_ids=terminators):
        print(f"{word}", end='', flush=True)
        output_text.append(word)
        total_tokens_count += n_tokens
    delta_time = time.time() - start_time
    output_text = "".join(output_text)
    print()
else:  
    start_time = time.time()
    outputs, total_tokens_count = generate_text(model, tokenizer, input_ids, max_gen_len=128, stop_tokens_ids=terminators)
    delta_time = time.time() - start_time
    print([text+output for text, output in zip(texts, outputs)])
print(f"Generation took {delta_time} seconds, {total_tokens_count/delta_time} tokens/s.")

```python
def find_max_value(numbers):
    max_value = numbers[0]
    for num in numbers:
        if num > max_value:
            max_value = num
    return max_value

numbers = [1, 5, 3, 7, 4, 8, 2]
print(find_max_value(numbers))
```
Generation took 9.491824865341187 seconds, 8.849720806240413 tokens/s.
